In [98]:
import numpy as np
import math
from scipy.special import kolmogorov


Колмогоров:

In [99]:
p = np.array([5,8,6,12,14,18,11,6,13,7])
F_emp = np.array([sum(p[:i]) for i in range(len(p)+1)]) / 100
F_th = np.arange(10) / 10

for j in range(10):
        sup = max(sup, abs(F_th[j] - F_emp[j]))
        sup = max(sup, abs(F_th[j] - F_emp[j + 1]))

delta = 10 * sup

kolmogorov(delta)


np.float64(0.039681879538114355)

вычисление дельта по ОМПГ

In [100]:
segments = np.array([(-np.inf, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, np.inf)])

sample = np.array([*[0] * 5, *[1] * 8, *[2] * 6, *[3]*12, *[4]*14,*[5]*18,*[6]*11,*[7]*6,*[8]*13,*[9]*7])

alpha = np.mean(sample)
disp = (np.var(sample) * 100 / 99) ** 0.5

def F_norm(x, mean1 = alpha, sigma1 = disp):
    return 0.5 * (1 + math.erf((x - mean1)/(np.sqrt(2) * sigma1)))

P = []

for i in segments:
    P.append(F_norm(i[1]) - F_norm(i[0]))

delta = sum((100 * P[i] - p[i]) ** 2 / 100 / P[i] for i in range(10))
delta

np.float64(16.871067048068724)

Вычисление p-value с помощью параметрического бутстрапа:

In [101]:
sample = np.array([*[0] * 5, *[1] * 8, *[2] * 6, *[3]*12, *[4]*14,*[5]*18,*[6]*11,*[7]*6,*[8]*13,*[9]*7])

alpha = np.mean(sample)
disp = (np.var(sample) * 100 / 99) ** 0.5

print(alpha, disp)

4.77 2.5180359509953494


In [102]:
iterations = 10000

def F_norm(x, mean1, sigma1):
    return 0.5 * (1 + math.erf((x - mean1)/(np.sqrt(2) * sigma1)))

p = np.array([5,8,6,12,14,18,11,6,13,7])
F_emp = np.array([sum(p[:i]) for i in range(len(p)+1)]) / 100
x = np.arange(10)
delta_wave = 10 * np.max([max(np.abs(F_norm(x[i], alpha, disp) - F_emp[i]),
                                      np.abs(F_norm(x[i], alpha, disp) - F_emp[i+1])) for i in range(10)])

sample_b = []

for i in range(iterations):
    sample = np.random.normal(alpha, disp, 100)
    sample = sorted(sample)
    sample_b.append(sample)



delta_z = []

for i in range(iterations):
    alpha_z = np.mean(sample_b[i])
    disp_z = (np.var(sample_b[i]) * 100 / 99) ** 0.5
    F = 0.0
    sup = 0

    for j in range(100):
        sup = max(sup, abs(F_norm(sample_b[i][j], alpha_z, disp_z) - F))
        F += 1/100
        sup = max(sup, abs(F_norm(sample_b[i][j], alpha_z, disp_z) - F))
    
    delta_z.append(10 * sup)
    
sum(delta_z >= delta_wave) / iterations

np.float64(0.0167)

Сравнение: отвергаем первую гипотезу, вторую тоже